In [2]:
import oracledb
import pandas as pd
import os
from datetime import datetime

def oracle_to_excel_oracledb():
    # Параметры подключения
    config = {
        'username': 'pim',
        'password': 'test123',
        'sql_query': """
with data as (
    select ctg_name, ctg_id, ctg_company_id, caa_value as spc_id, nod_id as root_nod
    from tctg_ctg_catalog 
        inner join TCTG_CAA_CATALOG_ATTRIBUTES on ctg_id = CAA_CATALOG_ID and CAA_NAME = 'SPEC_ID' and CAA_NEXT_VERSION_ID = 999999999 and ctg_company_id = CAA_COMPANY_ID         
        left join tctg_nod_node on NOD_SPEC_ID = caa_value and NOD_TYPE = 'ROOTNODE' and NOD_NEXT_VERSION_ID = 999999999
    where ctg_name in ('Позиции поставщика') and ctg_company_id = 2
)
------------------------ данные
    select
        i.itm_primary_key as "Код КССС"
        ,to_char(extractValue(cont.itx_entry_content,'//n2754')) as "Краткое наименование"
        ,to_char(extractValue(cont.itx_entry_content,'//n2766')) as "Полное наименование"
        ,to_char(n2851.itm_primary_key) as "Код базовой единицы измерения"
        ,to_char(extractValue(n2851c.itx_entry_content,'//n1328')) as "Базовая единица измерения"
        ,catatr.cab_value_string as "Код класса ТМЦ"
        ,catatr1.cab_value_string as "Наименование класса ТМЦ"
        ,to_char(extractValue(n2853c.itx_entry_content,'//n2340')) as "Статус записи"        
        ,to_char(extractValue(n2884c.itx_entry_content,'//n2310')) as "Тип записи"
        ,to_char(n2818.itm_primary_key) as "Код актуальной записи"
        ,to_char(extractValue(n2818c.itx_entry_content,'//n2754')) as "Актуальная запись"
        ,to_char(n2882.itm_primary_key) as "Код материала верхнего уровня"
        ,to_char(extractValue(n2882c.itx_entry_content,'//n2754')) as "Материал верхнего уровня"
        --,n2851c.itx_entry_content
        --,n2882.*
    from data nod 
        left join tctg_itm_item i on nod.ctg_id = i.itm_container_id and nod.ctg_company_id = i.itm_company_id and i.itm_next_version_id = 999999999 --and i.itm_primary_key = 23803
        left join tctg_itx_item_content cont on cont.itx_company_id = nod.ctg_company_id and cont.itx_container_id = nod.ctg_id and cont.itx_entry_id = i.itm_id 
            and i.itm_version_id = cont.itx_version_id
        left join tctg_icm_item_category_map cat on cat.icm_item_id = i.itm_id and cat.icm_cat_tree_id = 1602 and cat.icm_next_version_id = 999999999
        left join TCTG_CFP_CAT_FULL_PATHS p on p.cfp_category_id = cat.icm_category_id and p.cfp_next_version_id = 999999999
        left join TCTG_CAB_CATEGORY_ATTRIBUTES catatr on catatr.cab_category_id = cat.icm_category_id and catatr.cab_node_id = 1630 and catatr.cab_next_version_id = 999999999
        left join TCTG_CAB_CATEGORY_ATTRIBUTES catatr1 on catatr1.cab_category_id = cat.icm_category_id and catatr1.cab_node_id = 1618 and catatr1.cab_next_version_id = 999999999
        left join tctg_itm_item n2882 on n2882.itm_container_id  in (1839,1840) and nod.ctg_company_id = n2882.itm_company_id and n2882.itm_next_version_id = 999999999 and n2882.itm_primary_key = extractValue(cont.itx_entry_content,'//n2882')
        left join tctg_itx_item_content n2882c on n2882c.itx_company_id = nod.ctg_company_id and n2882c.itx_container_id = n2882.itm_container_id and n2882c.itx_entry_id = n2882.itm_id
            and 999999999 = n2882c.itx_next_version_id
            
        left join tctg_itm_item n2818 on n2818.itm_container_id in (1839,1840) and nod.ctg_company_id = n2818.itm_company_id and n2818.itm_next_version_id = 999999999 and n2818.itm_primary_key = extractValue(cont.itx_entry_content,'//n2818')
        left join tctg_itx_item_content n2818c on n2818c.itx_company_id = nod.ctg_company_id and n2818c.itx_container_id = n2818.itm_container_id and n2818c.itx_entry_id = n2818.itm_id
            and 999999999 = n2818c.itx_next_version_id         
            
        --Тип записи
        left join tctg_itm_item n2884 on n2884.itm_container_id in (1825) and nod.ctg_company_id = n2884.itm_company_id and n2884.itm_next_version_id = 999999999 and n2884.itm_id = to_char(extractValue(cont.itx_entry_content,'//n2884'))
        left join tctg_itx_item_content n2884c on n2884c.itx_company_id = nod.ctg_company_id and n2884c.itx_container_id = n2884.itm_container_id and n2884c.itx_entry_id = n2884.itm_id
            and 999999999 = n2884c.itx_next_version_id        
        -- Статус записи
        left join tctg_itm_item n2853 on n2853.itm_container_id in (1826) and nod.ctg_company_id = n2853.itm_company_id and n2853.itm_next_version_id = 999999999 and n2853.itm_id = to_char(extractValue(cont.itx_entry_content,'//n2853'))
        left join tctg_itx_item_content n2853c on n2853c.itx_company_id = nod.ctg_company_id and n2853c.itx_container_id = n2853.itm_container_id and n2853c.itx_entry_id = n2853.itm_id
            and 999999999 = n2853c.itx_next_version_id       
        --Базовая единица измерения
        left join tctg_itm_item n2851 on n2851.itm_container_id in (1418) and nod.ctg_company_id = n2851.itm_company_id and n2851.itm_next_version_id = 999999999 and n2851.itm_primary_key = extractValue(cont.itx_entry_content,'//n2851')
        left join tctg_itx_item_content n2851c on n2851c.itx_company_id = nod.ctg_company_id and n2851c.itx_container_id = n2851.itm_container_id and n2851c.itx_entry_id = n2851.itm_id
            and 999999999 = n2851c.itx_next_version_id
        """,
        'output_file': 'oracle_data.xlsx'
    }
    
    # Настройка thin mode (по умолчанию)
    #oracledb.init_oracle_client()  # Опционально, для thin mode
    
    try:
        # Создание строки подключения
        dsn = "ksss-prd-db01.srv.lukoil.com:1521/ksssprd1.world"
        
        print("Подключаемся к Oracle Database...")
        
        # Подключение к базе данных
        with oracledb.connect(
            user=config['username'],
            password=config['password'],
            dsn=dsn
        ) as connection:
            print("Успешное подключение к Oracle Database")
            
            # Выполнение запроса и создание DataFrame
            print("Выполняем SQL запрос...")
            df = pd.read_sql(config['sql_query'], connection)
            print(f"Загружено {len(df)} строк")
            
            # Сохранение в Excel файл
            df.to_excel(config['output_file'], index=False, sheet_name='Oracle_Data')
            print(f"Данные сохранены в файл: {config['output_file']}")
            
            # Показать информацию о данных
            print("\nИнформация о данных:")
            print(df.info())
            print(f"\nПервые 5 строк:")
            print(df.head())
            
    except oracledb.Error as error:
        print(f"Ошибка Oracle: {error}")
    except Exception as error:
        print(f"Общая ошибка: {error}")

# Запуск функции
if __name__ == "__main__":
    oracle_to_excel_oracledb()


Подключаемся к Oracle Database...
Успешное подключение к Oracle Database
Выполняем SQL запрос...


C:\Users\ZubarevVV\AppData\Local\Temp\ipykernel_14916\880076015.py:87: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(config['sql_query'], connection)


Загружено 913741 строк
Данные сохранены в файл: oracle_data.xlsx

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913741 entries, 0 to 913740
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   Код КССС                       913741 non-null  object
 1   Краткое наименование           913741 non-null  object
 2   Полное наименование            913741 non-null  object
 3   Код базовой единицы измерения  913741 non-null  object
 4   Базовая единица измерения      913741 non-null  object
 5   Код класса ТМЦ                 913737 non-null  object
 6   Наименование класса ТМЦ        913737 non-null  object
 7   Статус записи                  913741 non-null  object
 8   Тип записи                     913741 non-null  object
 9   Код актуальной записи          12316 non-null   object
 10  Актуальная запись              12316 non-null   object
 11  Код материала вер

In [5]:
import oracledb
import pandas as pd
from datetime import datetime
from typing import Tuple, Optional

def oracle_to_excel_with_dataframe(
    username: str,
    password: str,
    host: str,
    port: str,
    service_name: str,
    sql_query: str,
    output_file: str = None
) -> Tuple[pd.DataFrame, str]:
    """
    Подключается к Oracle, выполняет SQL запрос и возвращает DataFrame + сохраняет в Excel
    
    Args:
        username: Имя пользователя Oracle
        password: Пароль
        host: Хост сервера
        port: Порт
        service_name: Service name базы данных
        sql_query: SQL запрос для выполнения
        output_file: Путь для сохранения Excel (опционально)
    
    Returns:
        Tuple[DataFrame, str]: Датафрейм с данными и путь к сохраненному файлу
    """
    
    # Генерация имени файла если не указано
    if output_file is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_file = f"oracle_export_{timestamp}.xlsx"
    
    try:
        # Инициализация thin mode
        #oracledb.init_oracle_client()
        
        # Создание DSN
        dsn = f"{host}:{port}/{service_name}"
        
        print("Подключаемся к Oracle Database...")
        
        # Подключение к базе данных
        with oracledb.connect(
            user=username,
            password=password,
            dsn=dsn
        ) as connection:
            print("Успешное подключение к Oracle Database")
            
            # Выполнение запроса и создание DataFrame
            print("Выполняем SQL запрос...")
            df = pd.read_sql(sql_query, connection)
            print(f"Загружено {len(df)} строк, {len(df.columns)} колонок")
            
            # Сохранение в Excel файл
            df.to_excel(output_file, index=False, sheet_name='Oracle_Data')
            print(f"Данные сохранены в файл: {output_file}")
            
            # Возвращаем DataFrame и путь к файлу
            return df, output_file
            
    except oracledb.Error as error:
        print(f"Ошибка Oracle: {error}")
        # Возвращаем пустой DataFrame и None в случае ошибки
        return pd.DataFrame(), None
    except Exception as error:
        print(f"Общая ошибка: {error}")
        return pd.DataFrame(), None




In [10]:
# Пример использования ТМЦ И УСЛУГИ
if __name__ == "__main__":
    # Параметры подключения
    config = {
        'username': 'pim',
        'password': 'test123',
        'host': 'ksss-prd-db01.srv.lukoil.com',
        'port': '1521',
        'service_name': 'ksssprd1.world',
        'sql_query': """
with data as (
    select ctg_name, ctg_id, ctg_company_id, caa_value as spc_id, nod_id as root_nod
    from tctg_ctg_catalog 
        inner join TCTG_CAA_CATALOG_ATTRIBUTES on ctg_id = CAA_CATALOG_ID and CAA_NAME = 'SPEC_ID' and CAA_NEXT_VERSION_ID = 999999999 and ctg_company_id = CAA_COMPANY_ID         
        left join tctg_nod_node on NOD_SPEC_ID = caa_value and NOD_TYPE = 'ROOTNODE' and NOD_NEXT_VERSION_ID = 999999999
    where ctg_name in ('ТМЦ и услуги') and ctg_company_id = 2
)
------------------------ данные
    select
        i.itm_primary_key as "Код КССС"
        ,to_char(extractValue(cont.itx_entry_content,'//n2754')) as "Краткое наименование"
        ,to_char(extractValue(cont.itx_entry_content,'//n2766')) as "Полное наименование"
        ,to_char(n2851.itm_primary_key) as "Код базовой единицы измерения"
        ,to_char(extractValue(n2851c.itx_entry_content,'//n1328')) as "Базовая единица измерения"
        ,catatr.cab_value_string as "Код класса ТМЦ"
        ,catatr1.cab_value_string as "Наименование класса ТМЦ"
        ,to_char(extractValue(n2853c.itx_entry_content,'//n2340')) as "Статус записи"        
        ,to_char(extractValue(n2884c.itx_entry_content,'//n2310')) as "Тип записи"
        ,to_char(n2818.itm_primary_key) as "Код актуальной записи"
        ,to_char(extractValue(n2818c.itx_entry_content,'//n2754')) as "Актуальная запись"
        ,to_char(n2882.itm_primary_key) as "Код материала верхнего уровня"
        ,to_char(extractValue(n2882c.itx_entry_content,'//n2754')) as "Материал верхнего уровня"
        --,n2851c.itx_entry_content
        --,n2882.*
    from data nod 
        left join tctg_itm_item i on nod.ctg_id = i.itm_container_id and nod.ctg_company_id = i.itm_company_id and i.itm_next_version_id = 999999999 --and i.itm_primary_key = 23803
        left join tctg_itx_item_content cont on cont.itx_company_id = nod.ctg_company_id and cont.itx_container_id = nod.ctg_id and cont.itx_entry_id = i.itm_id 
            and i.itm_version_id = cont.itx_version_id
        left join tctg_icm_item_category_map cat on cat.icm_item_id = i.itm_id and cat.icm_cat_tree_id = 1602 and cat.icm_next_version_id = 999999999
        left join TCTG_CFP_CAT_FULL_PATHS p on p.cfp_category_id = cat.icm_category_id and p.cfp_next_version_id = 999999999
        left join TCTG_CAB_CATEGORY_ATTRIBUTES catatr on catatr.cab_category_id = cat.icm_category_id and catatr.cab_node_id = 1630 and catatr.cab_next_version_id = 999999999
        left join TCTG_CAB_CATEGORY_ATTRIBUTES catatr1 on catatr1.cab_category_id = cat.icm_category_id and catatr1.cab_node_id = 1618 and catatr1.cab_next_version_id = 999999999
        left join tctg_itm_item n2882 on n2882.itm_container_id  in (1839,1840) and nod.ctg_company_id = n2882.itm_company_id and n2882.itm_next_version_id = 999999999 and n2882.itm_primary_key = extractValue(cont.itx_entry_content,'//n2882')
        left join tctg_itx_item_content n2882c on n2882c.itx_company_id = nod.ctg_company_id and n2882c.itx_container_id = n2882.itm_container_id and n2882c.itx_entry_id = n2882.itm_id
            and 999999999 = n2882c.itx_next_version_id
            
        left join tctg_itm_item n2818 on n2818.itm_container_id in (1839,1840) and nod.ctg_company_id = n2818.itm_company_id and n2818.itm_next_version_id = 999999999 and n2818.itm_primary_key = extractValue(cont.itx_entry_content,'//n2818')
        left join tctg_itx_item_content n2818c on n2818c.itx_company_id = nod.ctg_company_id and n2818c.itx_container_id = n2818.itm_container_id and n2818c.itx_entry_id = n2818.itm_id
            and 999999999 = n2818c.itx_next_version_id         
            
        --Тип записи
        left join tctg_itm_item n2884 on n2884.itm_container_id in (1825) and nod.ctg_company_id = n2884.itm_company_id and n2884.itm_next_version_id = 999999999 and n2884.itm_id = to_char(extractValue(cont.itx_entry_content,'//n2884'))
        left join tctg_itx_item_content n2884c on n2884c.itx_company_id = nod.ctg_company_id and n2884c.itx_container_id = n2884.itm_container_id and n2884c.itx_entry_id = n2884.itm_id
            and 999999999 = n2884c.itx_next_version_id        
        -- Статус записи
        left join tctg_itm_item n2853 on n2853.itm_container_id in (1826) and nod.ctg_company_id = n2853.itm_company_id and n2853.itm_next_version_id = 999999999 and n2853.itm_id = to_char(extractValue(cont.itx_entry_content,'//n2853'))
        left join tctg_itx_item_content n2853c on n2853c.itx_company_id = nod.ctg_company_id and n2853c.itx_container_id = n2853.itm_container_id and n2853c.itx_entry_id = n2853.itm_id
            and 999999999 = n2853c.itx_next_version_id       
        --Базовая единица измерения
        left join tctg_itm_item n2851 on n2851.itm_container_id in (1418) and nod.ctg_company_id = n2851.itm_company_id and n2851.itm_next_version_id = 999999999 and n2851.itm_primary_key = extractValue(cont.itx_entry_content,'//n2851')
        left join tctg_itx_item_content n2851c on n2851c.itx_company_id = nod.ctg_company_id and n2851c.itx_container_id = n2851.itm_container_id and n2851c.itx_entry_id = n2851.itm_id
            and 999999999 = n2851c.itx_next_version_id
        """
    }

    # Вызов функции
    df_tmc, excel_file = oracle_to_excel_with_dataframe(output_file = 'TMC.xlsx',**config)
    
    # Работа с возвращенным DataFrame
    if not df_tmc.empty:
        print("\n" + "="*50)
        print("РАБОТА С ВОЗВРАЩЕННЫМ DATAFRAME:")
        print("="*50)
        
        # Основная информация о DataFrame
        print(f"Размер DataFrame: {df_tmc.shape}")
        print(f"Колонки: {list(df_tmc.columns)}")
        
        # Показать первые строки
        print("\nПервые 5 строк:")
        print(df_tmc.head())
        
        # Базовая статистика для числовых колонок
        numeric_columns = df_tmc.select_dtypes(include=['number']).columns
        if not numeric_columns.empty:
            print(f"\nСтатистика по числовым колонкам:")
            print(df_tmc[numeric_columns].describe())
        
        # Информация о файле
        print(f"\nФайл Excel сохранен: {excel_file}")
        
    else:
        print("Не удалось получить данные из базы")

Подключаемся к Oracle Database...
Успешное подключение к Oracle Database
Выполняем SQL запрос...


C:\Users\ZubarevVV\AppData\Local\Temp\ipykernel_14916\3123443485.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


Загружено 466621 строк, 13 колонок
Данные сохранены в файл: TMC.xlsx

РАБОТА С ВОЗВРАЩЕННЫМ DATAFRAME:
Размер DataFrame: (466574, 13)
Колонки: ['Код КССС', 'Краткое наименование', 'Полное наименование', 'Код базовой единицы измерения', 'Базовая единица измерения', 'Код класса ТМЦ', 'Наименование класса ТМЦ', 'Статус записи', 'Тип записи', 'Код актуальной записи', 'Актуальная запись', 'Код материала верхнего уровня', 'Материал верхнего уровня']

Первые 5 строк:
  Код КССС                     Краткое наименование  \
0    99999                 МТР 11 раздел регламента   
1   187288                          Нетопливные ТНП   
2  3547667                 Часть запорная к клапану   
3  3365907  Камера выносная для уровнемера VEGAFLEX   
4  1332732       Позиционер Norriseal 2SM60-TRDA-AB   

                                 Полное наименование  \
0                           МТР 11 раздел регламента   
1           Нетопливные товары народного потребления   
2            Часть запорная (седло+з

In [11]:
# Пример использования ПОЗИЦИИ ПОСТАВЩИКА
if __name__ == "__main__":
    # Параметры подключения
    config = {
        'username': 'pim',
        'password': 'test123',
        'host': 'ksss-prd-db01.srv.lukoil.com',
        'port': '1521',
        'service_name': 'ksssprd1.world',
        'sql_query': """
with data as (
    select ctg_name, ctg_id, ctg_company_id, caa_value as spc_id, nod_id as root_nod
    from tctg_ctg_catalog 
        inner join TCTG_CAA_CATALOG_ATTRIBUTES on ctg_id = CAA_CATALOG_ID and CAA_NAME = 'SPEC_ID' and CAA_NEXT_VERSION_ID = 999999999 and ctg_company_id = CAA_COMPANY_ID         
        left join tctg_nod_node on NOD_SPEC_ID = caa_value and NOD_TYPE = 'ROOTNODE' and NOD_NEXT_VERSION_ID = 999999999
    where ctg_name in ('Позиции поставщика') and ctg_company_id = 2
)
------------------------ данные
    select
        i.itm_primary_key as "Код КССС"
        ,to_char(extractValue(cont.itx_entry_content,'//n2754')) as "Краткое наименование"
        ,to_char(extractValue(cont.itx_entry_content,'//n2766')) as "Полное наименование"
        ,to_char(n2851.itm_primary_key) as "Код базовой единицы измерения"
        ,to_char(extractValue(n2851c.itx_entry_content,'//n1328')) as "Базовая единица измерения"
        ,catatr.cab_value_string as "Код класса ТМЦ"
        ,catatr1.cab_value_string as "Наименование класса ТМЦ"
        ,to_char(extractValue(n2853c.itx_entry_content,'//n2340')) as "Статус записи"        
        ,to_char(extractValue(n2884c.itx_entry_content,'//n2310')) as "Тип записи"
        ,to_char(n2818.itm_primary_key) as "Код актуальной записи"
        ,to_char(extractValue(n2818c.itx_entry_content,'//n2754')) as "Актуальная запись"
        ,to_char(n2882.itm_primary_key) as "Код материала верхнего уровня"
        ,to_char(extractValue(n2882c.itx_entry_content,'//n2754')) as "Материал верхнего уровня"
        --,n2851c.itx_entry_content
        --,n2882.*
    from data nod 
        left join tctg_itm_item i on nod.ctg_id = i.itm_container_id and nod.ctg_company_id = i.itm_company_id and i.itm_next_version_id = 999999999 --and i.itm_primary_key = 23803
        left join tctg_itx_item_content cont on cont.itx_company_id = nod.ctg_company_id and cont.itx_container_id = nod.ctg_id and cont.itx_entry_id = i.itm_id 
            and i.itm_version_id = cont.itx_version_id
        left join tctg_icm_item_category_map cat on cat.icm_item_id = i.itm_id and cat.icm_cat_tree_id = 1602 and cat.icm_next_version_id = 999999999
        left join TCTG_CFP_CAT_FULL_PATHS p on p.cfp_category_id = cat.icm_category_id and p.cfp_next_version_id = 999999999
        left join TCTG_CAB_CATEGORY_ATTRIBUTES catatr on catatr.cab_category_id = cat.icm_category_id and catatr.cab_node_id = 1630 and catatr.cab_next_version_id = 999999999
        left join TCTG_CAB_CATEGORY_ATTRIBUTES catatr1 on catatr1.cab_category_id = cat.icm_category_id and catatr1.cab_node_id = 1618 and catatr1.cab_next_version_id = 999999999
        left join tctg_itm_item n2882 on n2882.itm_container_id  in (1839,1840) and nod.ctg_company_id = n2882.itm_company_id and n2882.itm_next_version_id = 999999999 and n2882.itm_primary_key = extractValue(cont.itx_entry_content,'//n2882')
        left join tctg_itx_item_content n2882c on n2882c.itx_company_id = nod.ctg_company_id and n2882c.itx_container_id = n2882.itm_container_id and n2882c.itx_entry_id = n2882.itm_id
            and 999999999 = n2882c.itx_next_version_id
            
        left join tctg_itm_item n2818 on n2818.itm_container_id in (1839,1840) and nod.ctg_company_id = n2818.itm_company_id and n2818.itm_next_version_id = 999999999 and n2818.itm_primary_key = extractValue(cont.itx_entry_content,'//n2818')
        left join tctg_itx_item_content n2818c on n2818c.itx_company_id = nod.ctg_company_id and n2818c.itx_container_id = n2818.itm_container_id and n2818c.itx_entry_id = n2818.itm_id
            and 999999999 = n2818c.itx_next_version_id         
            
        --Тип записи
        left join tctg_itm_item n2884 on n2884.itm_container_id in (1825) and nod.ctg_company_id = n2884.itm_company_id and n2884.itm_next_version_id = 999999999 and n2884.itm_id = to_char(extractValue(cont.itx_entry_content,'//n2884'))
        left join tctg_itx_item_content n2884c on n2884c.itx_company_id = nod.ctg_company_id and n2884c.itx_container_id = n2884.itm_container_id and n2884c.itx_entry_id = n2884.itm_id
            and 999999999 = n2884c.itx_next_version_id        
        -- Статус записи
        left join tctg_itm_item n2853 on n2853.itm_container_id in (1826) and nod.ctg_company_id = n2853.itm_company_id and n2853.itm_next_version_id = 999999999 and n2853.itm_id = to_char(extractValue(cont.itx_entry_content,'//n2853'))
        left join tctg_itx_item_content n2853c on n2853c.itx_company_id = nod.ctg_company_id and n2853c.itx_container_id = n2853.itm_container_id and n2853c.itx_entry_id = n2853.itm_id
            and 999999999 = n2853c.itx_next_version_id       
        --Базовая единица измерения
        left join tctg_itm_item n2851 on n2851.itm_container_id in (1418) and nod.ctg_company_id = n2851.itm_company_id and n2851.itm_next_version_id = 999999999 and n2851.itm_primary_key = extractValue(cont.itx_entry_content,'//n2851')
        left join tctg_itx_item_content n2851c on n2851c.itx_company_id = nod.ctg_company_id and n2851c.itx_container_id = n2851.itm_container_id and n2851c.itx_entry_id = n2851.itm_id
            and 999999999 = n2851c.itx_next_version_id
        """
    }

    # Вызов функции
    df_sup, excel_file = oracle_to_excel_with_dataframe(output_file = 'SUP.xlsx',**config)
    
    # Работа с возвращенным DataFrame
    if not df_sup.empty:
        print("\n" + "="*50)
        print("РАБОТА С ВОЗВРАЩЕННЫМ DATAFRAME:")
        print("="*50)
        
        # Основная информация о DataFrame
        print(f"Размер DataFrame: {df_sup.shape}")
        print(f"Колонки: {list(df_sup.columns)}")
        
        # Показать первые строки
        print("\nПервые 5 строк:")
        print(df_sup.head())
        
        # Базовая статистика для числовых колонок
        numeric_columns = df_sup.select_dtypes(include=['number']).columns
        if not numeric_columns.empty:
            print(f"\nСтатистика по числовым колонкам:")
            print(df_sup[numeric_columns].describe())
        
        # Информация о файле
        print(f"\nФайл Excel сохранен: {excel_file}")
        
    else:
        print("Не удалось получить данные из базы")

Подключаемся к Oracle Database...
Успешное подключение к Oracle Database
Выполняем SQL запрос...


C:\Users\ZubarevVV\AppData\Local\Temp\ipykernel_14916\3123443485.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


Загружено 913891 строк, 13 колонок
Данные сохранены в файл: SUP.xlsx

РАБОТА С ВОЗВРАЩЕННЫМ DATAFRAME:
Размер DataFrame: (913891, 13)
Колонки: ['Код КССС', 'Краткое наименование', 'Полное наименование', 'Код базовой единицы измерения', 'Базовая единица измерения', 'Код класса ТМЦ', 'Наименование класса ТМЦ', 'Статус записи', 'Тип записи', 'Код актуальной записи', 'Актуальная запись', 'Код материала верхнего уровня', 'Материал верхнего уровня']

Первые 5 строк:
  Код КССС                  Краткое наименование  \
0   505470         _Задвижка 30нж41нж DN200 PN16   
1  1361456  Задвижка 30нж41нж DN200 PN16 газ КОФ   
2   506196     _Задвижка 30с941нж DN80 PN16, КОФ   
3   509901          _Задвижка 30с15нж DN100 PN40   
4   510466         _Задвижка 30нж15нж DN150 PN40   

                    Полное наименование Код базовой единицы измерения  \
0         _Задвижка 30нж41нж DN200 PN16                           108   
1  Задвижка 30нж41нж DN200 PN16 газ КОФ                            89   
2  

In [17]:
import oracledb
import openpyxl
from datetime import datetime
import json

class MainImproved:
    def __init__(self):
        self.host = "ksss-dev-db01.srv.lukoil.com"
        self.port = 1521
        self.service_name = "ksssdev1.world"
        self.login = "pim"
        self.password = "test123"
        
    def print_with_time(self, message):
        print(f"    {datetime.now().strftime('%H:%M:%S')}   {message}")
        
    def convert_lob_to_string(self, lob_data):
        """Конвертирует LOB объект в строку"""
        if lob_data is None:
            return ""
        if hasattr(lob_data, 'read'):
            try:
                content = lob_data.read()
                return str(content) if content is not None else ""
            except Exception as e:
                return f"[LOB read error: {str(e)}]"
        return str(lob_data)
        
    def get_sql(self, json_input):
        """Получает SQL из базы данных"""
        sql = ""
        connection = None
        try:
            dsn = f"{self.host}:{self.port}/{self.service_name}"
            connection = oracledb.connect(
                user=self.login,
                password=self.password,
                dsn=dsn
            )
            
            cursor = connection.cursor()
            
            # Выполняем запрос для получения SQL
            cursor.execute(f"select getCatalogSQL('{json_input}') from dual")
            result = cursor.fetchone()
            
            if result:
                sql = self.convert_lob_to_string(result[0])
                
            cursor.close()
            
        except Exception as e:
            self.print_with_time(f"SQL found error: {str(e)}")
        finally:
            if connection:
                connection.close()
                
        return sql
    
    def process_value(self, value):
        """Обрабатывает значение ячейки, конвертируя LOB в строку"""
        if value is None:
            return ""
        if hasattr(value, 'read'):  # Если это LOB
            return self.convert_lob_to_string(value)
        return str(value)
    
    def main(self):
        self.print_with_time("Start")
        
        # Формируем JSON для запроса SQL
        json_input = {
            "catalog": "КОНТРАГЕНТЫ",
            "attributes": [
                {"name": "CSCD_ID"},
                {"name": "Short_Name/ru_RU"},
                {"name": "Full_Name/ru_RU"},
                {"name": "INN"},
                {"name": "Full_Address/ru_RU"}
            ]
        }
        
        json_str = json.dumps(json_input, ensure_ascii=False)
        sql = self.get_sql(json_str)
        
        if not sql:
            self.print_with_time("SQL not found.")
            return
            
        self.print_with_time("SQL found.")
        
        connection = None
        cursor = None
        try:
            dsn = f"{self.host}:{self.port}/{self.service_name}"
            connection = oracledb.connect(
                user=self.login,
                password=self.password,
                dsn=dsn
            )
            
            cursor = connection.cursor()
            
            # Выполняем основной SQL запрос
            self.print_with_time("Executing main query...")
            cursor.execute(sql)
            
            self.print_with_time("ResultSet get.")
            
            # Получаем метаданные о колонках
            columns = [col[0] for col in cursor.description]
            column_count = len(columns)
            
            self.print_with_time("Export start.")
            
            # Создаем Excel workbook и sheet
            wb = openpyxl.Workbook()
            sheet = wb.active
            sheet.title = "Export KSSS"
            
            # Создаем заголовок
            sheet.append(columns)
            
            rownum = 1
            
            # Обрабатываем данные построчно
            batch_size = 1000
            while True:
                rows = cursor.fetchmany(batch_size)
                if not rows:
                    break
                
                for row_data in rows:
                    row_values = [self.process_value(value) for value in row_data]
                    sheet.append(row_values)
                    rownum += 1
                
                if rownum % 10000 == 0:
                    self.print_with_time(f"Export rowcount = {rownum}")
            
            self.print_with_time(f"Export end rowcount = {rownum}")
            
            # Сохраняем Excel файл
            try:
                output_file = "C:\\Users\\ZubarevVV\\Downloads\\template.xlsx"
                wb.save(output_file)
                self.print_with_time("Excel file created successfully.")
            except Exception as e:
                self.print_with_time(f"Excel create error: {str(e)}")
            
        except Exception as e:
            self.print_with_time(f"Error: {str(e)}")
            return
        finally:
            if cursor:
                cursor.close()
            if connection:
                connection.close()
                
        self.print_with_time("End")


if __name__ == "__main__":
    app = MainImproved()
    app.main()




    13:22:19   Start
    13:22:20   SQL found.
    13:22:20   Executing main query...
    13:22:20   ResultSet get.
    13:22:20   Export start.
    13:25:41   Export end rowcount = 383879
    13:25:59   Excel file created successfully.
    13:25:59   End


In [14]:
import oracledb
import openpyxl
from datetime import datetime
import json

class Main:
    def __init__(self):
        self.host = "ksss-dev-db01.srv.lukoil.com"
        self.port = 1521
        self.service_name = "ksssdev1.world"
        self.login = "pim"
        self.password = "test123"
        
    def print_with_time(self, message):
        print(f"    {datetime.now().strftime('%H:%M:%S')}   {message}")
        
    def get_sql(self, json_input):
        """Получает SQL из базы данных"""
        sql = ""
        connection = None
        try:
            # Создаем DSN строку
            dsn = f"{self.host}:{self.port}/{self.service_name}"
            
            # Устанавливаем соединение
            connection = oracledb.connect(
                user=self.login,
                password=self.password,
                dsn=dsn
            )
            
            cursor = connection.cursor()
            
            # Выполняем запрос для получения SQL
            cursor.execute(f"select getCatalogSQL('{json_input}') from dual")
            result = cursor.fetchone()
            
            if result:
                sql = result[0]
                
            cursor.close()
            
        except Exception as e:
            self.print_with_time(f"SQL found error: {str(e)}")
        finally:
            if connection:
                connection.close()
                
        return sql
    
    def main(self):
        self.print_with_time("Start")
        
        # Формируем JSON для запроса SQL (точно как в Java коде)
        json_input = {
            "catalog": "КОНТРАГЕНТЫ",
            "attributes": [
                {"name": "CSCD_ID"},
                {"name": "Short_Name/ru_RU"},
                {"name": "Full_Name/ru_RU"},
                {"name": "INN"},
                {"name": "Full_Address/ru_RU"}
            ]
        }
        
        json_str = json.dumps(json_input, ensure_ascii=False)
        sql = self.get_sql(json_str)
        
        if not sql:
            self.print_with_time("SQL not found.")
            return
            
        self.print_with_time("SQL found.")
        
        connection = None
        cursor = None
        try:
            # Устанавливаем соединение для основного запроса
            dsn = f"{self.host}:{self.port}/{self.service_name}"
            connection = oracledb.connect(
                user=self.login,
                password=self.password,
                dsn=dsn
            )
            
            cursor = connection.cursor()
            
            # Выполняем основной SQL запрос
            self.print_with_time("Executing main query...")
            cursor.execute(sql)
            
            self.print_with_time("ResultSet get.")
            
            # Получаем метаданные о колонках (аналог ResultSetMetaData)
            columns = [col[0] for col in cursor.description]
            column_count = len(columns)
            
            self.print_with_time("Export start.")
            
            # Создаем Excel workbook и sheet (аналог SXSSFWorkbook)
            wb = openpyxl.Workbook()
            sheet = wb.active
            sheet.title = "Export KSSS"
            
            # Создаем заголовок (первую строку)
            header_row = []
            for i in range(column_count):
                header_row.append(columns[i])
            sheet.append(header_row)
            
            rownum = 1  # Счетчик строк, начинаем с 1 т.к. заголовок уже добавлен
            
            # Обрабатываем данные построчно (аналог while (rs.next()))
            while True:
                row_data = cursor.fetchone()
                if row_data is None:
                    break
                
                # Создаем новую строку в Excel
                row_values = []
                for i in range(column_count):
                    value = row_data[i]
                    row_values.append(str(value) if value is not None else "")
                
                sheet.append(row_values)
                rownum += 1
                
                # Выводим прогресс каждые 10000 строк (как в Java коде)
                if rownum % 10000 == 0:
                    self.print_with_time(f"Export rowcount = {rownum}")
            
            self.print_with_time(f"Export end rowcount = {rownum}")
            
            # Сохраняем Excel файл
            try:
                output_file = "C:\\Users\\ZubarevVV\\Downloads\\template.xlsx"
                wb.save(output_file)
                self.print_with_time("Excel file created successfully.")
            except Exception as e:
                self.print_with_time(f"Excel create error: {str(e)}")
            
        except Exception as e:
            self.print_with_time(f"Error: {str(e)}")
            return
        finally:
            # Закрываем ресурсы (как в Java коде)
            if cursor:
                cursor.close()
            if connection:
                connection.close()
                
        self.print_with_time("End")


if __name__ == "__main__":
    # Если требуется Thick mode (раскомментировать при необходимости)
    # oracledb.init_oracle_client(lib_dir="/path/to/instant/client")
    
    app = Main()
    app.main()


    12:47:25   Start
    12:47:25   SQL found.
    12:47:25   Executing main query...
    12:47:25   Error: Argument 'statement' has incorrect type (expected str, got LOB)


In [18]:
2+2

4